# 📄 Amazon Bedrock + LangChain PDF Summarizer Demo
This notebook demonstrates how to summarize a PDF file using Amazon Bedrock with Claude and LangChain.

In [4]:
import boto3
from langchain.llms.bedrock import Bedrock
from langchain.chains.summarize import load_summarize_chain
from langchain.prompts import PromptTemplate
from langchain.docstore.document import Document
import PyPDF2

In [5]:
# --- CONFIGURATION ---
BEDROCK_REGION = "us-east-1"
MODEL_ID = "anthropic.claude-v2"

# --- INIT BEDROCK CLIENT ---
boto3_bedrock = boto3.client("bedrock-runtime", region_name=BEDROCK_REGION)
llm = Bedrock(model_id=MODEL_ID, client=boto3_bedrock)

/var/folders/dl/b4tc6ym136n02cybj804wy740000gp/T/ipykernel_11052/208283310.py:7: LangChainDeprecationWarning: The class `Bedrock` was deprecated in LangChain 0.0.34 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-aws package and should be used instead. To use it run `pip install -U :class:`~langchain-aws` and import as `from :class:`~langchain_aws import BedrockLLM``.
  llm = Bedrock(model_id=MODEL_ID, client=boto3_bedrock)


In [6]:
# --- FUNCTION: Load PDF and Extract Text ---
def extract_text_from_pdf(pdf_path):
    text = ""
    with open(pdf_path, "rb") as f:
        reader = PyPDF2.PdfReader(f)
        for page in reader.pages:
            text += page.extract_text()
    return text[:4000]  # Limit token length

In [17]:
# --- LOAD PDF ---
pdf_path = "sample1.pdf"
raw_text = extract_text_from_pdf(pdf_path)
document = Document(page_content=raw_text)

In [18]:
# --- PROMPT TEMPLATE ---
prompt_template = """Summarize the following content professionally:

{text}

Summary:
"""
prompt = PromptTemplate.from_template(prompt_template)
chain = load_summarize_chain(llm, chain_type="stuff", prompt=prompt)

In [19]:
# --- RUN SUMMARY ---
summary = chain.run([document])
print("\n📄 Document Summary:\n")
print(summary)


📄 Document Summary:

 Here is a professional summary of the key points:

The article discusses the application programming interface (API) management market and solutions. Gartner defines API management as software to manage, govern, and secure APIs that organizations use to modernize architectures and access systems, services, partners, and data. 

Key features of API management solutions include:

- API portal for discovery and trials
- API gateway for runtime management, security, throttling, monitoring
- Policy management for enforcement, mediation, limits, throttling, security
- Governance for version control, access, publication, operations

Additional common capabilities include API design, testing, monitoring, analytics, security, gateway federation, mediation, monetization, and AI. 

The article evaluates 17 vendors in Gartner's Magic Quadrant for API Management based on their ability to execute and completeness of vision. It notes strengths and weaknesses for Amazon Web Serv